In [5]:
# !pip install selenium

In [6]:
'''
1.需要提前打开手机12306二维码扫码工具
2.默认选择靠近过道的座位，若座位过少无法支持选座则该步操作忽略
3.仅支持购买二等座票
4.默认购买排序第一个人的票
'''

'\n1.需要提前打开手机12306二维码扫码工具\n2.默认选择靠近过道的座位，若座位过少无法支持选座则该步操作忽略\n3.仅支持购买二等座票\n4.默认购买排序第一个人的票\n'

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
from datetime import datetime

def retry_book_ticket(driver, target_train_number, max_attempts=30, refresh_interval=(0.5, 1.5)):
    for attempt in range(1, max_attempts + 1):
        try:
            WebDriverWait(driver, 2).until(
                EC.presence_of_element_located((By.ID, "queryLeftTable"))
            )
            book_buttons = driver.find_elements(By.XPATH,
                                                f"//tr[contains(., '{target_train_number}')]//a[contains(text(), '预订')]")

            if book_buttons:
                book_button = book_buttons[0]

            driver.execute_script(
                "arguments[0].scrollIntoView({block: 'center', inline: 'center'});",
                book_button
            )
            time.sleep(3)

            try:
                book_button.click()

                return f"成功预订{target_train_number}车次"
            except Exception as e:

                driver.execute_script("arguments[0].click();", book_button)

                return f"成功预订{target_train_number}车次"

        except Exception as e:
            print(f"第{attempt}次尝试失败: {str(e)}")

            if attempt < max_attempts:
                try:
                    refresh_btn = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.ID, "query_ticket"))
                    )
                    refresh_btn.click()

                    wait_time = random.uniform(*refresh_interval)
                    print(f"等待{wait_time:.2f}秒后进行下一次尝试...")
                    time.sleep(wait_time)

                except Exception as refresh_err:
                    print(f"刷新页面失败: {str(refresh_err)}，尝试直接刷新页面...")
                    driver.refresh()
                    time.sleep(random.uniform(*refresh_interval))

    return f"没抢到，可惜~"

def select_seat_fast(driver, preferred_type="过道"):
    print(f"快速选择座位，首选类型: {preferred_type}")

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "seat-sel-bd"))
        )
        print("座位选择对话框已加载")
    except Exception as e:
        print(f"座位选择对话框加载失败: {str(e)}")
        return False

    # 直接获取所有可用座位
    try:
        seats = driver.find_elements(By.XPATH, "//div[@class='seat-sel-bd']//a[contains(@href, 'javascript:')]")
        print(f"找到 {len(seats)} 个可用座位")
        
        if not seats:
            return False

        # 如果有座位，直接选择第一个（最快的方式）
        seats[0].click()
        print("快速选择第一个可用座位")
        return True
        
    except Exception as e:
        print(f"快速选座失败: {str(e)}")
        return False

In [8]:
if __name__ == "__main__":
    
    #提前输入购买信息
    start_position = "龙川西"  #出发站(需要完整的名字)
    reach_position = "河源东"  #到达站(需要完整的名字)
    start_time = "2025-10-05"  #出发日期，需要写成标准时间格式如示例
    kind = 1     #0是学生票，1是普通票
    target_train_number = "G6541"   #要购买的趟次

    booking_start_time = "2025-9-22 21:23:00"  #购票时间，是你要抢票的时间，不是出发时间
    print(f"抢票任务设置: {start_position} -> {reach_position} {start_time} {target_train_number}")
    print(f"抢票开始时间: {booking_start_time}")
    print(f"请扫码登录，需要提前开启手机12306软件的扫码工具")


    from selenium.webdriver.edge.options import Options

    edge_options = Options()
    edge_options.add_experimental_option("detach", True)
    edge_options.add_argument("--disable-blink-features=AutomationControlled")
    edge_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36 Edg/140.0.3485.54")
    driver = webdriver.Edge(options=edge_options)
    driver.get('https://www.12306.cn')
    driver.maximize_window()

    try:
        login_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.ID, 'J-btn-login'))
        )
        login_button.click()

    except Exception as e:
        print(f"点击登录按钮失败：{str(e)}")
        driver.quit()
        exit()

    try:
        scan_login_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//a[text()='扫码登录']"))
        )
        scan_login_button.click()
        print("已切换到扫码登录，请用手机12306扫码...")

    except Exception as e:
        print(f"点击扫码登录按钮失败：{str(e)}")
        driver.quit()
        exit()

    login_success = False
    print("等待扫码登录...")
    for _ in range(30):
        try:
            WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, "//a[text()='个人中心']"))
            )
            login_success = True
            break
        except:
            time.sleep(1)

    if not login_success:
        print("登录超时或未成功，请重新尝试")
        driver.quit()
        exit()
        
    try:
        # 进入购票页面
        ticket_link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'link_for_ticket'))
        )
        ticket_link.click()
        print("已进入购票页面")

    except Exception as e:
        print(f"进入购票页面失败：{str(e)}")
        driver.quit()
        exit()

    try:
        # 操作出发地输入框
        from_station_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'fromStationText'))
        )
        from_station_input.click()
        from_station_input.clear()
        from_station_input.send_keys(start_position)
        print(f"已输入出发地: {start_position}")

        first_option = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '#citem_0 > span:nth-child(1)'))
        )
        first_option.click()

    except Exception as e:
        print(f"操作出发地输入框失败：{str(e)}")

        exit()

    try:
        # 操作目的地输入框
        to_station_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'toStationText'))
        )
        to_station_input.click()
        to_station_input.clear()
        to_station_input.send_keys(reach_position)
        print(f"已输入目的地: {reach_position}")

        # 选择第一个选项
        first_option = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '#citem_0 > span:nth-child(1)'))
        )
        first_option.click()

    except Exception as e:
        print(f"操作目的地输入框失败：{str(e)}")

        exit()

    try:
        # 操作日期输入框
        date_input = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.ID, 'train_date'))
        )
        date_input.click()
        date_input.clear()
        date_input.send_keys(start_time)
        print(f"已输入出发时间: {start_time}")


        driver.find_element(By.CLASS_NAME, 'cal').click()

    except Exception as e:
        print(f"时间输入框操作失败：{str(e)}")

        exit()

    try:
        if kind == 0:
            # 学生票
            student_ticket = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.ID, 'sf2'))
            )
            student_ticket.click()
            print("已选择学生票")
        else:
            # 普通票
            normal_ticket = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.ID, 'sf1'))
            )
            normal_ticket.click()
            print("已选择普通票")
    except Exception as e:
        print(f"票种选择失败：{str(e)}")
        exit()

    try:
        # 转换为datetime对象
        start_datetime = datetime.strptime(booking_start_time, "%Y-%m-%d %H:%M:%S")
        now = datetime.now()
        
        if now < start_datetime:
            # 计算等待时间（秒）
            wait_seconds = (start_datetime - now).total_seconds()
            print(f"距离抢票开始还有 {wait_seconds:.1f} 秒，等待中...")
            
            if wait_seconds > 10:
                time.sleep(wait_seconds - 10)
            
            # 精确等待到预定时间
            while datetime.now() < start_datetime:
                time.sleep(0.1)
        
        print("到达抢票时间，开始抢票！")
    except Exception as e:
        print(f"时间处理出错: {str(e)}")
        driver.quit()
        exit()

    try:
        # 点击查询按钮
        query_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.ID, 'query_ticket'))
        )
        query_button.click()
        print("已提交查询，正在等待结果...")
        time.sleep(1)  # 等待查询结果加载

    except Exception as e:
        print(f"查询失败：{str(e)}")

        exit()
        
    print(f"开始尝试预订 {target_train_number} 次列车...")
    result = retry_book_ticket(
        driver=driver,
        target_train_number=target_train_number,
        max_attempts=30,
        refresh_interval=(3, 6)
    )

    try:
        passenger_checkbox = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "normalPassenger_0"))
        )
        passenger_checkbox.click()
        print("已成功选择乘车人")
    except Exception as e:
        print(f"选择乘车人失败：{str(e)}")
    try:
        confirm_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "dialog_xsertcj_ok"))
        )
        confirm_button.click()
        print("已成功点击提示框的确认按钮")
    except Exception as e:
        print(f"点击确认按钮失败：{str(e)}")

    if kind == 1:
        try:
            # 定位票种选择下拉框（根据你提供的HTML，id为ticketType_1）
            ticket_type_select = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "ticketType_1"))
            )
            
            # 创建Select对象
            from selenium.webdriver.support.ui import Select
            select = Select(ticket_type_select)
            
            select.select_by_value("1")
            print("已成功选择票种：成人票")
            
        except Exception as e:
            print(f"选择票种失败：{str(e)}")


        
    try:
        # 定位“提交订单”按钮（通过id属性精准定位）
        submit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "submitOrder_id"))
        )

        submit_button.click()
        print("已成功点击提交订单按钮")
         
    except Exception as e:
        print(f"点击提交订单按钮失败：{str(e)}")
    time.sleep(1)

    if kind == 0:
        try:
            confirm_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "qd_closeDefaultWarningWindowDialog_id"))
            )
    
            confirm_button.click()
            print("已成功点击提示框的确认按钮")
        except Exception as e:
            print(f"点击确认按钮失败：{str(e)}")
            
    select_seat_fast(driver, preferred_type="过道")
    time.sleep(1.5)

    try:
        confirm_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "qr_submit_id"))
        )

        confirm_button.click()

    except Exception as e:
        print(f"点击确认按钮失败：{str(e)}")

抢票任务设置: 龙川西 -> 河源东 2025-10-05 G6541
抢票开始时间: 2025-9-22 21:23:00
请扫码登录，需要提前开启手机12306软件的扫码工具
已切换到扫码登录，请用手机12306扫码...
等待扫码登录...
已进入购票页面
已输入出发地: 龙川西
已输入目的地: 河源东
已输入出发时间: 2025-10-05
已选择普通票
到达抢票时间，开始抢票！
已提交查询，正在等待结果...
开始尝试预订 G6541 次列车...
已成功选择乘车人
已成功点击提示框的确认按钮
已成功选择票种：成人票
已成功点击提交订单按钮
快速选择座位，首选类型: 过道
座位选择对话框已加载
找到 34 个可用座位
快速选座失败: Message: element not interactable
  (Session info: MicrosoftEdge=140.0.3485.81); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementnotinteractableexception
Stacktrace:
	GetHandleVerifier [0x0x7ff600d43f15+18309]
	(No symbol) [0x0x7ff600cb15c0]
	(No symbol) [0x0x7ff600ab0df0]
	(No symbol) [0x0x7ff600aff742]
	(No symbol) [0x0x7ff600af4bbe]
	(No symbol) [0x0x7ff600b1fc7a]
	(No symbol) [0x0x7ff600af443b]
	(No symbol) [0x0x7ff600af42ed]
	(No symbol) [0x0x7ff600b1ff30]
	(No symbol) [0x0x7ff600af443b]
	(No symbol) [0x0x7ff600b3d082]
	(No symbol) [0x0x7ff600b1f943]
	(No symbol) [0x0x7ff600af38e6]
	(